In [2]:
import pandas as pd
import numpy as np
import datetime
import os

In [3]:
# --- 0. Configuración Global ---
DAY_OF_REFERENCE = pd.to_datetime('2022-12-31') # Día de referencia para los cálculos de antigüedad y actividad

print(f"Calculando KPIs con día de referencia: {DAY_OF_REFERENCE.strftime('%Y-%m-%d')}\n")

# ==============================================================================
# --- 1. Cargar y Preprocesar los DataFrames individuales ---
# ==============================================================================

# 1.1 Cargar y preprocesar 2025-07 purchases_challenge
purchase_date_cols = [
    'Signup Date',
    'Last Login Date',
    'First Order Date',
    'Last Order Date'
]
try:
    # Asegúrate de que este sea el nombre correcto de tu archivo
    df_purchases = pd.read_csv('2025-07 purchases_challenge.csv', parse_dates=purchase_date_cols)
    print("DataFrame '2025-07 purchases_challenge.csv' cargado y fechas parseadas correctamente.")
    
    # Asegurarse de que 'Signup Year' esté presente para el cálculo del slider (si aplica)
    # Aunque no se use el slider aquí, es bueno tener la columna si se generó antes.
    if 'Signup Year' not in df_purchases.columns:
        df_purchases['Signup Year'] = df_purchases['Signup Date'].dt.year

except FileNotFoundError:
    print("ERROR: '2025-07 purchases_challenge.csv' no encontrado. Asegúrese de que el archivo existe en el directorio correcto.")
    exit()
except Exception as e:
    print(f"ERROR al cargar '2025-07 purchases_challenge.csv': {e}")
    exit()

# 1.2 Cargar y preprocesar 2025-07 refunds_challenge
# Asumo que 2025-07 refunds_challenge.csv tiene 'Member ID', una columna para la cantidad de reembolsos ('Refunds')
# y una columna para el monto reembolsado ('Credit Amount').
# AJUSTA 'refund_count_column' y 'refund_amount_column' si los nombres son diferentes en tu archivo.
refund_count_column = 'Refunds'
refund_amount_column = 'Credit Amount'
refund_date_cols = [] # Añade aquí si 2025-07 refunds_challenge tiene columnas de fecha relevantes, ej: ['Refund Date']

try:
    # Asegúrate de que este sea el nombre correcto de tu archivo
    df_refunds = pd.read_csv('2025-07 refunds_challenge.csv', parse_dates=refund_date_cols)
    print("DataFrame '2025-07 refunds_challenge.csv' cargado.")
    
    # --- Agrupar reembolsos por Member ID para unirlos al df_purchases ---
    # Sumar el número de reembolsos por cliente (si un Member ID puede tener múltiples filas de reembolso)
    # y el monto total reembolsado.
    
    # Verificar si las columnas existen antes de intentar agrupar
    if refund_count_column in df_refunds.columns and refund_amount_column in df_refunds.columns:
        df_refunds_summary = df_refunds.groupby('Member ID').agg(
            Total_Refund_Count=(refund_count_column, 'sum'),
            Total_Refund_Amount=(refund_amount_column, 'sum')
        ).reset_index()
    else:
        # Si las columnas no existen, crea un DataFrame de resumen con ceros
        print(f"ADVERTENCIA: Las columnas '{refund_count_column}' y/o '{refund_amount_column}' no se encontraron en '2025-07 refunds_challenge.csv'. Las tasas de reembolso se establecerán en 0.")
        df_refunds_summary = pd.DataFrame({'Member ID': df_purchases['Member ID'].unique(), 
                                           'Total_Refund_Count': 0, 
                                           'Total_Refund_Amount': 0})


except FileNotFoundError:
    print("ADVERTENCIA: '2025-07 refunds_challenge.csv' no encontrado. Los cálculos de tasa de reembolso se basarán en 0 reembolsos.")
    df_refunds_summary = pd.DataFrame({'Member ID': df_purchases['Member ID'].unique(), 
                                       'Total_Refund_Count': 0, 
                                       'Total_Refund_Amount': 0})
except Exception as e:
    print(f"ERROR inesperado al cargar o procesar '2025-07 refunds_challenge.csv': {e}. Los cálculos de tasa de reembolso se basarán en 0 reembolsos.")
    df_refunds_summary = pd.DataFrame({'Member ID': df_purchases['Member ID'].unique(), 
                                       'Total_Refund_Count': 0, 
                                       'Total_Refund_Amount': 0})

# ==============================================================================
# --- 2. Unir los DataFrames para un análisis completo ---
# ==============================================================================
# Unir df_purchases con la información resumida de reembolsos.
# Usaremos 'left merge' para mantener todos los clientes de df_purchases y añadirles la info de reembolso.
# Los clientes sin correspondencia en df_refunds_summary tendrán NaN, que rellenaremos con 0.
df_combined = pd.merge(df_purchases, df_refunds_summary, on='Member ID', how='left')
df_combined[['Total_Refund_Count', 'Total_Refund_Amount']] = df_combined[['Total_Refund_Count', 'Total_Refund_Amount']].fillna(0)

print("\n--- Vista previa del DataFrame combinado (primeras 5 filas) ---")
print(df_combined.head())
print("\n--- Información del DataFrame combinado ---")
print(df_combined.info())
print("\n" + "="*50 + "\n")


# ==============================================================================
# --- 3. Calcular los 10 Indicadores de Rendimiento (KPIs) ---
# ==============================================================================

# 1. Período mínimo de pre-compra.
# 2. Período máximo de pre-compra.
# Definición: Tiempo entre la fecha de registro (Signup Date) y la primera orden (First Order Date).
# Filtramos para asegurar que First Order Date no sea nulo y que sea posterior o igual a Signup Date.
df_prepurchase_calc = df_combined.dropna(subset=['Signup Date', 'First Order Date']).copy()
df_prepurchase_calc = df_prepurchase_calc[df_prepurchase_calc['First Order Date'] >= df_prepurchase_calc['Signup Date']]

if not df_prepurchase_calc.empty:
    pre_purchase_period = (df_prepurchase_calc['First Order Date'] - df_prepurchase_calc['Signup Date']).dt.days
    min_pre_purchase = pre_purchase_period.min()
    max_pre_purchase = pre_purchase_period.max()
    print(f"1. Período mínimo de pre-compra: {min_pre_purchase} días")
    print(f"2. Período máximo de pre-compra: {max_pre_purchase} días")
else:
    print("No hay datos válidos para calcular el período de pre-compra (First Order Date >= Signup Date).")
print("-" * 30)


# Definición de clientes "activos" y "cancelados" (churned)
# Consideraremos un cliente "activo" si tuvo actividad (Last Login Date o Last Order Date)
# en los últimos 365 días previos al día de referencia (2022-12-31).
active_threshold_date = DAY_OF_REFERENCE - pd.Timedelta(days=365) # Esto es 2021-12-31

df_combined['Is_Active'] = (
    ((df_combined['Last Login Date'].notna()) & (df_combined['Last Login Date'] >= active_threshold_date)) |
    ((df_combined['Last Order Date'].notna()) & (df_combined['Last Order Date'] >= active_threshold_date))
)

active_customers_df = df_combined[df_combined['Is_Active'] == True].copy()
canceled_customers_df = df_combined[df_combined['Is_Active'] == False].copy()

print(f"Total de clientes activos (con actividad desde {active_threshold_date.strftime('%Y-%m-%d')}): {len(active_customers_df)}")
print(f"Total de clientes cancelados (sin actividad desde {active_threshold_date.strftime('%Y-%m-%d')}): {len(canceled_customers_df)}")
print("-" * 30)


# 3. Antigüedad promedio de los clientes activos.
# Antigüedad = DÍA DE REFERENCIA - Fecha de registro (Signup Date)
if not active_customers_df.empty:
    active_tenure = (DAY_OF_REFERENCE - active_customers_df['Signup Date']).dt.days
    avg_active_tenure = active_tenure.mean()
    print(f"3. Antigüedad promedio de los clientes activos: {avg_active_tenure:.2f} días")
else:
    print("No hay clientes activos para calcular la antigüedad promedio.")

# 4. Antigüedad promedio de los clientes cancelados.
if not canceled_customers_df.empty:
    canceled_tenure = (DAY_OF_REFERENCE - canceled_customers_df['Signup Date']).dt.days
    avg_canceled_tenure = canceled_tenure.mean()
    print(f"4. Antigüedad promedio de los clientes cancelados: {avg_canceled_tenure:.2f} días")
else:
    print("No hay clientes cancelados para calcular la antigüedad promedio.")
print("-" * 30)


# 5. Número promedio de pedidos realizados por clientes cancelados.
if not canceled_customers_df.empty:
    avg_orders_canceled = canceled_customers_df['Purchases'].mean()
    print(f"5. Número promedio de pedidos realizados por clientes cancelados: {avg_orders_canceled:.2f}")
else:
    print("No hay clientes cancelados para calcular el número promedio de pedidos.")

# 6. Número promedio de pedidos realizados por clientes activos.
if not active_customers_df.empty:
    avg_orders_active = active_customers_df['Purchases'].mean()
    print(f"6. Número promedio de pedidos realizados por clientes activos: {avg_orders_active:.2f}")
else:
    print("No hay clientes activos para calcular el número promedio de pedidos.")
print("-" * 30)


# 7. Monto promedio gastado por clientes cancelados.
if not canceled_customers_df.empty:
    avg_amount_canceled = canceled_customers_df['Total Amount'].mean()
    print(f"7. Monto promedio gastado por clientes cancelados: {avg_amount_canceled:.2f}")
else:
    print("No hay clientes cancelados para calcular el monto promedio gastado.")

# 8. Monto promedio gastado por clientes activos.
if not active_customers_df.empty:
    avg_amount_active = active_customers_df['Total Amount'].mean()
    print(f"8. Monto promedio gastado por clientes activos: {avg_amount_active:.2f}")
else:
    print("No hay clientes activos para calcular el monto promedio gastado.")
print("-" * 30)


# 9. Tasa de reembolso basada en el número de pedidos.
# Tasa de reembolso por pedidos = (Total de reembolsos) / (Total de pedidos) * 100
total_purchases_count = df_combined['Purchases'].sum()
total_refund_events = df_combined['Total_Refund_Count'].sum() # Usa la columna sumada de reembolsos

if total_purchases_count > 0:
    refund_rate_by_orders = (total_refund_events / total_purchases_count) * 100
    print(f"9. Tasa de reembolso basada en el número de pedidos: {refund_rate_by_orders:.2f}%")
else:
    print("No hay pedidos para calcular la tasa de reembolso por número de pedidos.")

# 10. Tasa de reembolso basada en el monto gastado.
# Tasa de reembolso por monto = (Monto total reembolsado) / (Monto total gastado) * 100
total_amount_spent = df_combined['Total Amount'].sum()
total_amount_refunded = df_combined['Total_Refund_Amount'].sum() # Usa la columna sumada de monto reembolsado

if total_amount_spent > 0:
    refund_rate_by_amount = (total_amount_refunded / total_amount_spent) * 100
    print(f"10. Tasa de reembolso basada en el monto gastado: {refund_rate_by_amount:.2f}%")
else:
    print("No hay monto gastado para calcular la tasa de reembolso por monto.")
print("-" * 30)

Calculando KPIs con día de referencia: 2022-12-31

DataFrame '2025-07 purchases_challenge.csv' cargado y fechas parseadas correctamente.
DataFrame '2025-07 refunds_challenge.csv' cargado.

--- Vista previa del DataFrame combinado (primeras 5 filas) ---
   Member ID Signup Date Last Login Date  Purchases  Total Amount  \
0     289584  2015-01-01      2021-01-22          1        401.51   
1     289586  2015-01-01      2020-06-14          1         89.48   
2     289592  2015-01-01      2023-05-01          1        128.03   
3     289599  2015-01-01      2023-05-29          4       2724.27   
4     289616  2015-01-01      2023-05-10          4       2125.65   

   Credit Amount First Order Date Last Order Date  Signup Year  \
0            NaN       2016-12-18      2016-12-18         2015   
1            NaN       2020-01-30      2020-01-30         2015   
2            NaN       2018-12-08      2018-12-08         2015   
3            NaN       2019-01-29      2019-11-08         2015   
4 

In [ ]:
Explicación del Código y lo que te revelará:

Carga de DataFrames: Intenta cargar purchases_df_processed.csv y refunds_df_processed.csv. Si alguno no se encuentra, imprimirá un error y detendrá la ejecución del análisis.

Análisis de Member ID en purchases_df:

df_purchases['Member ID'].duplicated().any(): Esto verifica rápidamente si hay algún Member ID que aparezca más de una vez en el DataFrame de compras.

Si hay duplicados, te mostrará las primeras 10 filas donde Member ID está duplicado (con keep=False para mostrar todas las ocurrencias del ID duplicado), lo cual es crucial. Si Member ID está duplicado en purchases_df, esto significa que cada fila podría ser una transacción individual de un cliente, no el resumen de su actividad. Este es un punto muy importante para la agregación antes de calcular KPIs.

Análisis de Member ID en refunds_df:

df_refunds['Member ID'].duplicated().any(): Similar al anterior, verifica si hay Member IDs que aparezcan más de una vez en el DataFrame de reembolsos.

Si True: Esto es lo que esperas si un cliente puede tener múltiples reembolsos. Significa que cada fila en refunds_df es probablemente un evento de reembolso individual (o una transacción que contiene un reembolso), en lugar de un resumen consolidado por cliente.

El código entonces utiliza value_counts() para contar cuántas veces aparece cada Member ID.

Luego filtra aquellos Member IDs que tienen un conteo mayor a 1, y te muestra ejemplos.

Finalmente, te muestra un ejemplo de todas las filas para uno de esos Member ID con múltiples reembolsos, para que veas la granularidad de los datos.

Comparación de Member ID únicos entre DataFrames:

Calcula el número de Member ID únicos en cada archivo.

intersection(): Te dice cuántos clientes están presentes en ambos archivos (es decir, tienen registros de compras Y de reembolsos).

difference(): Te indica cuántos clientes solo están en un archivo y no en el otro. Esto es útil para saber, por ejemplo, cuántos clientes compraron pero nunca tuvieron un reembolso, o si hay registros de reembolso para clientes de los que no tienes un historial de compra principal.

Qué buscar en la salida:

En purchases_df: Si ¿Hay 'Member ID' duplicados en purchases_df_processed.csv? es True, significa que tu purchases_df no es un resumen de clientes, sino un registro de transacciones o eventos. En ese caso, para los KPIs, necesitarías groupby('Member ID') y sum() o mean() las columnas relevantes (Purchases, Total Amount, etc.) para obtener un resumen por cliente antes de calcular los KPIs que son "por cliente".

En refunds_df: Si ¿Hay 'Member ID' duplicados en refunds_df_processed.csv? es True, esto confirma que tu refunds_df registra reembolsos individuales. El código que te di para los KPIs (en la respuesta anterior) ya maneja esto agrupando df_refunds por Member ID y sumando Total_Refund_Count y Total_Refund_Amount, lo cual es el enfoque correcto para este escenario.

En la sección de comparación: Te dará una idea de la superposición de tus bases de clientes.

Ejecuta este código y analiza la salida. Si los Member ID están repetidos en purchases_df_processed.csv, la forma en que calculamos los KPIs necesitaría un paso adicional de groupby('Member ID') en df_purchases al inicio para obtener un resumen por cliente único antes de proceder.